In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
import os
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from keras.models import load_model
from nilearn import datasets, plotting
from scipy.ndimage import zoom

In [ ]:
def resize(file, shape):
    img = nib.load(file)
    data = img.get_fdata()

    # Definir o novo formato de imagem
    new_shape = shape
    zoom_factors = np.array(new_shape) / np.array(data.shape)

    # order 0 é para manter os valores
    resized_data = zoom(data, zoom_factors, order=0)  
    cropped_data = resized_data[24:169, 24:206, 6:161]
    return cropped_data

def plot_image_mask(patient_data, mask_data):
    # Número de fatias que você deseja plotar
    num_slices = 7

    # Criar subplots em uma única linha
    fig, axes = plt.subplots(1, num_slices, figsize=(20, 5))  # Ajuste o tamanho conforme necessário

    # Iterar por cada fatia z
    for z in range(0, num_slices, 1):
        ax = axes[z]  # Acessa o subplot para a fatia z

        # Exibir a imagem da fatia do paciente em escala de cinza
        ax.imshow(patient_data[:, :, z], cmap='gray')

        # Sobrepor a máscara do atlas com transparência ajustada
        ax.imshow(mask_data[:, :, z], cmap='nipy_spectral', alpha=0.7)

        ax.set_title(f"z = {z}")
        ax.axis("off")  # Desativa os eixos

    # Ajustar o layout para que as imagens não se sobreponham
    plt.tight_layout()
    plt.show()

def save_useless_labels(labels, atlas):
    to_remove = [0]

    for idx in range(0, len(labels)):
        nulo = np.count_nonzero(atlas == idx)
        if nulo == 0:
            to_remove.append(idx)

    print(f"Número de labels nos slices definidos: {len(labels)}\nNúmero de labels sem valor nos slices definidos: {len(to_remove)}")

    '''for item in to_remove:
        if item in labels:
            labels.remove(item)

    print(f"Número de labels depois da remoção: {len(labels)}\n")'''
    return to_remove

def plot_each_region(atlas_data, patient_data, labels, ignore):
    # Máscara por intervalo
    for idx in range(0, len(labels)):
        if idx not in (ignore):
            fig, axes = plt.subplots(1, 7, figsize=(20, 5))
            i = 0

            for z in range(0, 7):
                atlas_slice = atlas_data[:, :, z]
                patient_slice = patient_data[:, :, z]

                # Criar uma máscara binária para exibir apenas os labels dentro do intervalo
                mask = np.isin(atlas_slice, idx)

                # Criar uma cópia do atlas onde apenas os valores do label são mantidos
                filtered_atlas = np.zeros_like(atlas_slice)
                filtered_atlas[mask] = atlas_slice[mask]  # Mantém só os valores do label

                # Exibir as imagens lado a lado
                axes[i].imshow(patient_slice, cmap='gray')
                axes[i].imshow(filtered_atlas, cmap='jet', alpha=0.5) # o alpha é a transparência
                axes[i].set_title(f"z = {z}")
                axes[i].axis("off")
                i += 1 

            plt.suptitle(f"Grupo: {labels[idx]}", fontsize=16)
            plt.subplots_adjust(top=1.3)  # Ajusta o espaço entre título e imagens
            plt.show()

In [ ]:
raw_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/raw_data/3D_BRAIN_NOT_NORMALIZED/test/ad/I297850.nii.gz"
patient_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/test/ad/I300337.nii.gz"
data_patient = nib.load(raw_path).get_fdata()

In [ ]:
# Carregar o atlas Harvard-Oxford cortical e subcortical
atlas_cortical = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_img_cortical = nib.load(atlas_cortical.filename)
data_cortical = atlas_img_cortical.get_fdata()

atlas_subcortical = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
atlas_img_subcortical = nib.load(atlas_subcortical.filename)
data_subcortical = atlas_img_subcortical.get_fdata()

In [ ]:
# Visualizar o atlas cortical
plotting.plot_roi(atlas_cortical.maps, title="Harvard-Oxford Cortical Atlas")

# Visualizar o atlas subcortical
plotting.plot_roi(atlas_subcortical.maps, title="Harvard-Oxford Subcortical Atlas")

plotting.show()

cortical_labels = atlas_cortical.labels
subcortical_labels = atlas_subcortical.labels

# Listar os nomes das regiões
print("Regiões Corticais:")
for idx, region in enumerate(cortical_labels):
    print(f"{idx}: {region}")

print("\nRegiões Subcorticais:")
for idx, region in enumerate(subcortical_labels):
    print(f"{idx}: {region}")


In [ ]:
for idx, region in enumerate(cortical_labels):
    indice_cortical = idx 

    # Criar uma máscara para o índice da região
    mask = np.zeros_like(data_cortical)
    mask[data_cortical == indice_cortical] = 1

    # Criar uma imagem de máscara a partir dos dados
    mask_img = nib.Nifti1Image(mask, atlas_img_cortical.affine)

    # Plotar a região
    plotting.plot_roi(mask_img, title=f"Região {region} - Cortical", display_mode='z', draw_cross=True)

    plotting.show()

In [ ]:
for idx in range(1, len(subcortical_labels) - 1//2):

    indice_subcortical = idx
    mask_subcortical = np.zeros_like(data_subcortical)
    mask_subcortical[(data_subcortical == indice_subcortical) | (data_subcortical == indice_subcortical+11)] = 1  # Criar máscara para o índice

    # Criar imagem de máscara subcortical
    mask_img_subcortical = nib.Nifti1Image(mask_subcortical, atlas_img_subcortical.affine)

    # Plotar a região subcortical
    plotting.plot_roi(mask_img_subcortical, title=f"Região {region} - Subcortical", display_mode='z', draw_cross=False)

    # Mostrar os gráficos
    plotting.show()

In [ ]:
print(data_cortical.shape)
plt.imshow(data_cortical[:, :, 50])
plt.show()

print(data_patient.shape)

In [ ]:
new_shape = (193, 229, 193)

resized_cortical_data = resize(atlas_cortical.filename, new_shape)
resized_subcortical_data = resize(atlas_subcortical.filename, new_shape)

plt.imshow(resized_cortical_data[:, :, 88])
plt.show()

plt.imshow(resized_subcortical_data[:, :, 88])
plt.show()

##### ALINHAMENTO DA MÁSCARA COM MRI

In [ ]:
# harvard-oxford -> [17:162, 19:201, 0:155]
# mri -> [24:169, 24:206, 6:161]
# aalr3 -> [21:166, 18:200, 0:155]

lim = resized_cortical_data.shape[2]

slices = (60, 68, 76, 82, 89, 92, 96)

data_cortical_slices = resized_cortical_data[:, :, slices]
data_subcortical_slices = resized_subcortical_data[:, :, slices]
data_patient_slices = data_patient[:, :, slices]

In [ ]:
# Listar os nomes das regiões
print("Regiões Corticais:")
for idx, region in enumerate(cortical_labels):
    print(f"{idx}: {region}")

print("\nRegiões Subcorticais:")
for idx, region in enumerate(subcortical_labels):
    print(f"{idx}: {region}")

In [ ]:
cortical_ignore = save_useless_labels(cortical_labels, data_cortical_slices)
subcortical_ignore = save_useless_labels(subcortical_labels, data_subcortical_slices)

In [ ]:
# Listar os nomes das regiões
print("Regiões Corticais:")
for idx, region in enumerate(cortical_labels):
    print(f"{idx}: {region}")

print("\nRegiões Subcorticais:")
for idx, region in enumerate(subcortical_labels):
    print(f"{idx}: {region}")

In [ ]:
plot_image_mask(data_patient_slices, data_cortical_slices)

In [ ]:
plot_image_mask(data_patient_slices, data_subcortical_slices)

In [ ]:
print(f"LABELS CORTICAL ATLAS ({len(cortical_labels)}):\n{cortical_labels}\n\nLABELS SUBCORTICAL ATLAS ({len(subcortical_labels)}):\n{subcortical_labels}")

In [ ]:
patient_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/test/ad/I300337.nii.gz"
patient_pred = nib.load(patient_path).get_fdata()

In [ ]:
plot_each_region(data_cortical_slices, patient_pred, cortical_labels, cortical_ignore)

In [ ]:
plot_each_region(data_subcortical_slices, patient_pred, subcortical_labels, subcortical_ignore)

##### PLOT MÁSCARA POR REGIÃO

In [ ]:
cortical_dict = {}

print("Regiões Corticais:")
for idx, region in enumerate(cortical_labels):
    print(f"{idx}: {region}")

# Máscara por intervalo
for idx in range(0, len(cortical_labels)):
    if idx not in cortical_ignore:
        fig, axes = plt.subplots(1, 7, figsize=(20, 5))
        final_mask = data_cortical_slices.copy()
        i = 0

        for z in range(0, 7):
            atlas_slice = data_cortical_slices[:, :, z]
            patient_slice = patient_pred[:, :, z]

            mask = np.isin(atlas_slice, idx).astype(np.uint8) 

            final_mask[:, :, z] = mask

            # Exibir as imagens lado a lado
            axes[i].imshow(patient_slice, cmap='gray')
            axes[i].imshow(mask, cmap='jet', alpha=0.5) # o alpha é a transparência
            axes[i].set_title(f"z = {slices[z]}")
            axes[i].axis("off")
            i += 1

        cortical_dict[cortical_labels[idx]] = final_mask

        plt.suptitle(f"Grupo: {cortical_labels[idx]}", fontsize=16)
        plt.subplots_adjust(top=1.3)  # Ajusta o espaço entre título e imagens
        plt.show()

In [ ]:
subcortical_dict = {}
n_labels_sub = len(subcortical_labels)

print("\nRegiões Subcorticais:")
for idx, region in enumerate(subcortical_labels):
    print(f"{idx}: {region}")

# Máscara por intervalo
for idx in range(0, (n_labels_sub+1)//2+1):
    if idx not in subcortical_ignore:
        name = subcortical_labels[idx]
        if idx != 8:
            name = " ".join(name.split()[1:])  

        fig, axes = plt.subplots(1, 7, figsize=(20, 5))
        final_mask = data_subcortical_slices.copy()
        i = 0

        for z in range(0, 7):
            atlas_slice = data_subcortical_slices[:, :, z]
            patient_slice = patient_pred[:, :, z]

            if idx < 8:
                mask = np.isin(atlas_slice, (idx, (idx + (n_labels_sub+1)//2))).astype(np.uint8)
            elif idx == 8:
                mask = np.isin(atlas_slice, (idx)).astype(np.uint8)
            else:
                mask = np.isin(atlas_slice, (idx, (idx - 1 + (n_labels_sub+1)//2))).astype(np.uint8)

            final_mask[:, :, z] = mask

            # Exibir as imagens lado a lado
            axes[i].imshow(patient_slice, cmap='gray')
            axes[i].imshow(mask, cmap='jet', alpha=0.5) # o alpha é a transparência
            axes[i].set_title(f"z = {slices[z]}")
            axes[i].axis("off")
            i += 1

        subcortical_dict[name] = final_mask

        if idx == 8:
            plt.suptitle(f"Grupo {idx}: {name}", fontsize=16)
        else: 
            plt.suptitle(f"Grupo {idx} e {(idx - 1 + (n_labels_sub+1)//2)}: {name}", fontsize=16)
        plt.subplots_adjust(top=1.3)  # Ajusta o espaço entre título e imagens
        plt.show()

In [ ]:
print(cortical_dict.keys())

if 'Cerebral White Matter' in subcortical_dict:
    subcortical_dict.pop('Cerebral White Matter')
if 'Cerebral Cortex' in subcortical_dict:
    subcortical_dict.pop('Cerebral Cortex')

print(subcortical_dict.keys())

In [ ]:
for name in cortical_dict.keys():
    mascara = cortical_dict[name]
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))
    
    for z in range(0, 7):
        patient_slice = patient_pred[:, :, z]
        mask = mascara[:, :, z]

        axes[z].imshow(patient_slice, cmap='gray')
        axes[z].imshow(mask, alpha=0.5)
        axes[z].axis('off')
        axes[z].set_title(f"z = {slices[z]}")
        
    plt.suptitle(name)
    plt.subplots_adjust(top=1.3)
    plt.show()

In [ ]:
for name in subcortical_dict.keys():
    mascara = subcortical_dict[name]
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))
    
    for z in range(0, 7):
        patient_slice = patient_pred[:, :, z]
        mask = mascara[:, :, z]

        axes[z].imshow(patient_slice, cmap='gray')
        axes[z].imshow(mask, alpha=0.5)
        axes[z].axis('off')
        axes[z].set_title(f"z = {slices[z]}")
        
    plt.suptitle(name)
    plt.subplots_adjust(top=1.3)
    plt.show()

##### TESTAR OCLUSÃO

In [ ]:
for name in subcortical_dict.keys():
    mascara = subcortical_dict[name]
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))

    masked_patient = (1-mascara) * patient_pred
    
    for z in range(0, 7):
        patient_slice = masked_patient[:, :, z]

        axes[z].imshow(patient_slice, cmap='gray')
        axes[z].axis('off')
        axes[z].set_title(f"z = {slices[z]}")
        
    plt.suptitle(f"Subcortical - {name}")
    plt.subplots_adjust(top=1.3)
    plt.show()

for name in cortical_dict.keys():
    mascara = cortical_dict[name]
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))

    masked_patient = (1-mascara) * patient_pred
    
    for z in range(0, 7):
        patient_slice = masked_patient[:, :, z]

        axes[z].imshow(patient_slice, cmap='gray')
        axes[z].axis('off')
        axes[z].set_title(f"z = {slices[z]}")
        
    plt.suptitle(f"Cortical - {name}")
    plt.subplots_adjust(top=1.3)
    plt.show()

##### PREDIZER OCLUSÃO

In [ ]:
model_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/results/3d/axial_certo/test_5/binary_classifier_120_epochs_batch_64_5_classes.keras"
model = tf.keras.models.load_model(model_path)

# Suponha que `data_patient` seja o MRI original
original_image = np.expand_dims(patient_pred, axis=(0, -1))  # Adiciona batch e canal (1, H, W, D, 1)

# Fazer a predição na imagem original
original_pred = model.predict(original_image)[0]
print(original_pred)

# Armazenar resultados
importance_map = {}

# Iterar sobre cada versão ocluída do dicionário
for region_name, occluded_mri in mri_masked_dict.items():
    print(region_name)
    occluded_image = np.expand_dims(occluded_mri, axis=(0, -1))  # Adicionar batch e canal

    # Fazer predição na versão ocluída
    occluded_pred = model.predict(occluded_image)[0]
    print(occluded_pred)

    # Calcular a diferença na predição
    importance = np.abs(original_pred - occluded_pred)

    # Armazenar a importância da região
    importance_map[region_name] = importance

    #print(f"Região: {region_name} → Impacto na predição: {importance}")

print(f"IMPORTANCIA:\n")
for item in mri_masked_dict:
    print(f"{item}: {importance_map[item]}")

##### PREDIZER TESTE INTEIRO

In [ ]:
def load_data(path, classes):
    dataset = []
    labels = []

    for i in range(0, len(classes)):
        dir = f"{path}/{classes[i]}"

        for data in os.listdir(dir):
            data_path = f"{dir}/{data}"
            image = nib.load(data_path).get_fdata()
            image = image[..., np.newaxis]
            dataset.append(image)
            labels.append(i)

    return np.array(dataset), np.array(labels)

def occlude_data(data, mask):
    mask = mask[..., np.newaxis]

    occluded = np.copy(data)
    
    for i in range(0, len(occluded)):
            occluded[i] = occluded[i] * mask

    return occluded

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

def predict_results(data, labels, model):
    y_pred_original = (model.predict(data))

    original_class = np.argmax(y_pred_original, axis=1)

    print(classification_report(labels, original_class))

    plot_confusion_matrix(labels, original_class)

In [ ]:
base_dir = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial"

test_dir = f"{base_dir}/test"
model_path = f"{base_dir}/results/3d/axial_certo/test_5/binary_classifier_120_epochs_batch_64_5_classes.keras"


classes = ['cn', 'emci', 'mci', 'lmci', 'ad']

In [ ]:
data, labels = load_data(test_dir, classes)

model = load_model(model_path)

predict_results(data, labels, model)